In [1]:
# Cell 1: imports
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import RobustScaler

import lightgbm as lgb
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Cell 2: load processed data
TRAIN_PATH = "/kaggle/input/processed-financial-risk/train_processed.csv"
TEST_PATH  = "/kaggle/input/processed-financial-risk/test_processed.csv"
SAMPLE_PATH = "/kaggle/input/financial-risk-profiling/sample_submission_updated.csv"

train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)
sample = pd.read_csv(SAMPLE_PATH)

print("Train shape:", train.shape)
print("Test shape :", test.shape)

Train shape: (204277, 27)
Test shape : (51070, 26)


In [3]:
# Cell 3: prepare X, y, X_test (drop ProfileID)
IDCOL = "ProfileID"
TARGET = "RiskFlag"

X = train.drop(columns=[IDCOL, TARGET])
y = train[TARGET].astype(int)

X_test = test.drop(columns=[IDCOL])
test_ids = test[IDCOL].copy()

print("X shape:", X.shape, "y shape:", y.shape, "X_test shape:", X_test.shape)

X shape: (204277, 25) y shape: (204277,) X_test shape: (51070, 25)


In [4]:
# ===============================
# Cell A: Clustering Feature Engineering (Safe)
# ===============================

from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
import numpy as np

# Make copies
X_cluster = X.copy()
X_test_cluster = X_test.copy()

# ===============================
# 1. KMEANS CLUSTERS
# ===============================
K = 8
kmeans = KMeans(n_clusters=K, random_state=42, n_init="auto")
kmeans.fit(X)

# hard labels
X_cluster["kmeans_label"] = kmeans.labels_
X_test_cluster["kmeans_label"] = kmeans.predict(X_test)

# cluster distances (soft-ish)
km_dist_train = kmeans.transform(X)
km_dist_test = kmeans.transform(X_test)

for i in range(K):
    X_cluster[f"kmeans_dist_{i}"] = km_dist_train[:, i]
    X_test_cluster[f"kmeans_dist_{i}"] = km_dist_test[:, i]

print("✔ KMeans features added.")

# ===============================
# 2. GAUSSIAN MIXTURE MODEL (GMM)
# ===============================
G = 6
gmm = GaussianMixture(
    n_components=G,
    covariance_type="full",
    random_state=42
)

gmm.fit(X)

# soft probabilities
gmm_train = gmm.predict_proba(X)
gmm_test = gmm.predict_proba(X_test)

for i in range(G):
    X_cluster[f"gmm_prob_{i}"] = gmm_train[:, i]
    X_test_cluster[f"gmm_prob_{i}"] = gmm_test[:, i]

print("✔ GMM probability features added.")

# ===============================
# Final assignment
# ===============================
X = X_cluster
X_test = X_test_cluster

print("🎉 All clustering features successfully integrated!")


✔ KMeans features added.
✔ GMM probability features added.
🎉 All clustering features successfully integrated!


In [5]:
# ===============================
# Cell B: Replace X & X_test with clustered versions
# ===============================

X = X_cluster
X_test = X_test_cluster

print("New X shape:", X.shape)
print("New X_test shape:", X_test.shape)

New X shape: (204277, 40)
New X_test shape: (51070, 40)


In [6]:
# Cell 4: stacking containers
NFOLDS = 5
kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=42)

oof_lr = np.zeros(len(X))
oof_lgb = np.zeros(len(X))
oof_xgb = np.zeros(len(X))
oof_cat = np.zeros(len(X))

test_lr = np.zeros(len(X_test))
test_lgb = np.zeros(len(X_test))
test_xgb = np.zeros(len(X_test))
test_cat = np.zeros(len(X_test))

In [7]:
# Cell 5: base models definitions

# Logistic Regression (base)
lr_model_template = LogisticRegression(
    max_iter=2000,
    solver="lbfgs",
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)

# LightGBM
lgb_params = dict(
    n_estimators=800,
    learning_rate=0.05,
    num_leaves=64,
    max_depth=-1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)
lgb_model_template = lgb.LGBMClassifier(**lgb_params)

# XGBoost
xgb_params = dict(
    n_estimators=800,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)
xgb_model_template = XGBClassifier(**xgb_params)

# CatBoost
cat_model_template = CatBoostClassifier(
    iterations=800,
    learning_rate=0.05,
    depth=6,
    random_seed=42,
    verbose=0
)

In [8]:
# Cell 6: helper - scaler to use only for logistic regression (not for tree models)
scaler = RobustScaler()

In [9]:
# ===============================
# Cell 7: OOF Stacking (LR + LGB + XGB + Cat + PyTorch MLP)
# ===============================

import numpy as np
from sklearn.metrics import roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim

oof_lr  = np.zeros(len(X))
oof_lgb = np.zeros(len(X))
oof_xgb = np.zeros(len(X))
oof_cat = np.zeros(len(X))
oof_mlp = np.zeros(len(X))

test_lr  = np.zeros(len(X_test))
test_lgb = np.zeros(len(X_test))
test_xgb = np.zeros(len(X_test))
test_cat = np.zeros(len(X_test))
test_mlp = np.zeros(len(X_test))

# ----------------------
# PyTorch MLP definition
# ----------------------
class MLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

def train_mlp(X_tr, y_tr, X_val):
    X_tr = torch.tensor(X_tr.values, dtype=torch.float32)
    y_tr = torch.tensor(y_tr.values.reshape(-1, 1), dtype=torch.float32)
    X_val = torch.tensor(X_val.values, dtype=torch.float32)

    model = MLP(X_tr.shape[1])
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.BCELoss()

    for epoch in range(15):
        optimizer.zero_grad()
        preds = model(X_tr)
        loss = loss_fn(preds, y_tr)
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        val_preds = model(X_val).numpy().flatten()

    return model, val_preds

# ----------------------
# K-Fold Loop
# ----------------------
for fold, (train_idx, valid_idx) in enumerate(kf.split(X, y), 1):
    print(f"\n=== FOLD {fold} ===")

    X_tr, X_val = X.iloc[train_idx], X.iloc[valid_idx]
    y_tr, y_val = y.iloc[train_idx], y.iloc[valid_idx]

    # ------- Logistic Regression -------
    X_tr_s = scaler.fit_transform(X_tr)
    X_val_s = scaler.transform(X_val)
    lr = LogisticRegression(**lr_model_template.get_params())
    lr.fit(X_tr_s, y_tr)

    oof_lr[valid_idx] = lr.predict_proba(X_val_s)[:, 1]
    test_lr += lr.predict_proba(scaler.transform(X_test))[:, 1] / NFOLDS

    print("LR AUC:", roc_auc_score(y_val, oof_lr[valid_idx]))

    # ------- LightGBM -------
    lgbm = lgb.LGBMClassifier(**lgb_params)
    lgbm.fit(X_tr, y_tr)

    oof_lgb[valid_idx] = lgbm.predict_proba(X_val)[:, 1]
    test_lgb += lgbm.predict_proba(X_test)[:, 1] / NFOLDS

    print("LGB AUC:", roc_auc_score(y_val, oof_lgb[valid_idx]))

    # ------- XGBoost -------
    xgb = XGBClassifier(**xgb_params)
    xgb.fit(X_tr, y_tr)

    oof_xgb[valid_idx] = xgb.predict_proba(X_val)[:, 1]
    test_xgb += xgb.predict_proba(X_test)[:, 1] / NFOLDS

    print("XGB AUC:", roc_auc_score(y_val, oof_xgb[valid_idx]))

    # ------- CatBoost -------
    cat = CatBoostClassifier(**cat_model_template.get_params())
    cat.fit(X_tr, y_tr, verbose=False)

    oof_cat[valid_idx] = cat.predict_proba(X_val)[:, 1]
    test_cat += cat.predict_proba(X_test)[:, 1] / NFOLDS

    print("CAT AUC:", roc_auc_score(y_val, oof_cat[valid_idx]))

    # ------- PyTorch MLP -------
    mlp_model, val_mlp = train_mlp(X_tr, y_tr, X_val)
    oof_mlp[valid_idx] = val_mlp

    with torch.no_grad():
        test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
        test_mlp += mlp_model(test_tensor).numpy().flatten() / NFOLDS

    print("MLP AUC:", roc_auc_score(y_val, oof_mlp[valid_idx]))


=== FOLD 1 ===
LR AUC: 0.74429613159447
LGB AUC: 0.7415415017932456
XGB AUC: 0.7427148751185825
CAT AUC: 0.749128319656637
MLP AUC: 0.5

=== FOLD 2 ===
LR AUC: 0.7510840842384393
LGB AUC: 0.7472615741747511
XGB AUC: 0.7481266654746492
CAT AUC: 0.7561576327411759
MLP AUC: 0.5

=== FOLD 3 ===
LR AUC: 0.7479524783225856
LGB AUC: 0.7472724987644954
XGB AUC: 0.7457689409010835
CAT AUC: 0.7541885301174369
MLP AUC: 0.5

=== FOLD 4 ===
LR AUC: 0.7459546883146768
LGB AUC: 0.7431133345567139
XGB AUC: 0.7418578053135494
CAT AUC: 0.7515997564890144
MLP AUC: 0.5

=== FOLD 5 ===
LR AUC: 0.7423435270537652
LGB AUC: 0.7403451544727752
XGB AUC: 0.7397206269093745
CAT AUC: 0.7491741472765736
MLP AUC: 0.5


In [10]:
# Cell 8: OOF AUC scores
print("OOF LR AUC :", roc_auc_score(y, oof_lr))
print("OOF LGB AUC:", roc_auc_score(y, oof_lgb))
print("OOF XGB AUC:", roc_auc_score(y, oof_xgb))
print("OOF CAT AUC:", roc_auc_score(y, oof_cat))

OOF LR AUC : 0.746306080083499
OOF LGB AUC: 0.7438813985166557
OOF XGB AUC: 0.7436194108725395
OOF CAT AUC: 0.7520392088112163


In [11]:
# Cell 9: stacking datasets
stack_train = pd.DataFrame({
    "lr": oof_lr,
    "lgb": oof_lgb,
    "xgb": oof_xgb,
    "cat": oof_cat
})

stack_test = pd.DataFrame({
    "lr": test_lr,
    "lgb": test_lgb,
    "xgb": test_xgb,
    "cat": test_cat
})

print("stack_train shape:", stack_train.shape)
print("stack_test shape :", stack_test.shape)
stack_train.head()

stack_train shape: (204277, 4)
stack_test shape : (51070, 4)


,lr,lgb,xgb,cat
0,0.727058,0.059886,0.183968,0.136221
1,0.616267,0.061471,0.104382,0.109813
2,0.809898,0.295425,0.241453,0.222791
3,0.400829,0.053788,0.044667,0.098383
4,0.480894,0.184791,0.174229,0.101012


In [12]:
# Cell 10: meta-model training
meta = LogisticRegression(max_iter=2000, solver="lbfgs", random_state=42)
meta.fit(stack_train, y)

meta_oof = meta.predict_proba(stack_train)[:, 1]
print("META OOF AUC:", roc_auc_score(y, meta_oof))

META OOF AUC: 0.7517269415087152


In [13]:
# Cell 11: meta predict on test, then threshold to 0/1
final_probs = meta.predict_proba(stack_test)[:, 1]
final_preds = (final_probs >= 0.5).astype(int)

# Quick sanity
print("Final probs sample:", final_probs[:6])
print("Final preds sample:", final_preds[:6])

Final probs sample: [0.03825348 0.04825424 0.07892736 0.09020903 0.11713769 0.15309723]
Final preds sample: [0 0 0 0 0 0]


In [14]:
# Cell 12: make submission (0/1)
submission = sample.copy()
submission["RiskFlag"] = final_preds
OUT_PATH = "logreg_nn_lgb_xgb_cat_stacked.csv"
submission.to_csv(OUT_PATH, index=False)
OUT_PATH

'logreg_nn_lgb_xgb_cat_stacked.csv'